In [9]:
import PyPDF2
import csv
import json
import pandas as pd
import matplotlib.pyplot as plt
import camelot
import tabula
import requests
import os
import re

In [168]:
def download(url, file_name):
    # open in binary mode
    with open(file_name, "wb") as file:
        # get request
        response = requests.get(url)
        # write to file
        file.write(response.content)
#iterate over df and access links, download pdfs, convert tables to csv
def get_tables(df,start=0, end=-1):
    tables = []
    for row in df[start:end].values:
        for i in row:
            if '.pdf' in str(i):
                if '200' in str(requests.get(str(i))):
                    download(str(i), str(row[1])+'.pdf')
                    pages, names = lookup_table(str(row[1])+'.pdf')
                    tables += tabula.read_pdf(str(row[1])+'.pdf',multiple_tables=True, pages=pages)
                    x,y=1,0
                    for t in tables:
                        t.columns = t.iloc[0]
                        t.drop(t.index[0], inplace=True)
                        t['jurisdiction'] = str(row[1])
                        t['location_type'] = str(row[2])
                        t['table_number'] = x
                        t['table_name'] = names[y]
                        x += 1
                        y += 1
                    return tables
#                     tables = tabula.convert_into(str(row[1])+'.pdf', str(row[1])+'.csv',output_format='csv',
#                                              multiple_tables=True, pages='all')
#                     os.remove(str(row[1])+'.pdf')
def lookup_table(pdf_doc):
    string = r'([T|t]{1}able\s[\w+\d+].+)\n'
    doc = PyPDF2.PdfFileReader(open(pdf_doc, 'rb'))
    pages = []
    names = []
    for i in range(doc.getNumPages()):
        page = doc.getPage(i)
        text = page.extractText()
        if re.search(string, text):
            pages.append(i)
            names += re.findall(r'[T|t]{1}able\s[\d+\w+][:|\.|\n](.+)\n', text)
    return pages, names

In [169]:
#Load list of urls to extract pdfs from
df = pd.read_csv('Cities.csv')
a = get_tables(df,16,20)

In [170]:
a

[0                                                 NaN  Cumulative  \
 1                                                 NaN         GHG   
 2                                            Strategy  Reductions   
 3                                                 NaN  (MT CO2e),   
 4                                                 NaN   2012–2020   
 5   Use Biomethane in CNG Buses (well-to-wheels im...   528,555**   
 6             On-board Railcar Braking Energy Storage      96,411   
 7   Gasoline-Electric Hybrid Buses (tank-to-wheels...      76,826   
 8             Building Indoor Lighting Upgrades: LEDs      71,621   
 9   Building Indoor Lighting Upgrades: Efficient M...      46,226   
 10           Wayside Energy Storage Substation (WESS)      17,289   
 11               Retrofit Lighting in Red Line Tunnel       5,783   
 12                     Expand Use of Renewable Energy       4,467   
 13           Municipal Recycled Water For Bus Washing         941   
 14          Extensi